    Problem Statement.
    
    Table: Accounts
    
    +---------------+---------+
    | Column Name   | Type    |
    +---------------+---------+
    | id            | int     |
    | name          | varchar |
    +---------------+---------+
    id is the primary key (column with unique values) for this table.
    This table contains the account id and the user name of each account.
    
     
    
    Table: Logins
    
    +---------------+---------+
    | Column Name   | Type    |
    +---------------+---------+
    | id            | int     |
    | login_date    | date    |
    +---------------+---------+
    This table may contain duplicate rows.
    This table contains the account id of the user who logged in and the login date. A user may log in multiple times in the day.
    
     
    
    Active users are those who logged in to their accounts for five or more consecutive days.
    
    Write a solution to find the id and the name of active users.
    
    Return the result table ordered by id.
    
    The result format is in the following example.
    
     
    
    Example 1:
    
    Input: 
    Accounts table:
    +----+----------+
    | id | name     |
    +----+----------+
    | 1  | Winston  |
    | 7  | Jonathan |
    +----+----------+
    Logins table:
    +----+------------+
    | id | login_date |
    +----+------------+
    | 7  | 2020-05-30 |
    | 1  | 2020-05-30 |
    | 7  | 2020-05-31 |
    | 7  | 2020-06-01 |
    | 7  | 2020-06-02 |
    | 7  | 2020-06-02 |
    | 7  | 2020-06-03 |
    | 1  | 2020-06-07 |
    | 7  | 2020-06-10 |
    +----+------------+
    Output: 
    +----+----------+
    | id | name     |
    +----+----------+
    | 7  | Jonathan |
    +----+----------+
    Explanation: 
    User Winston with id = 1 logged in 2 times only in 2 different days, so, Winston is not an active user.
    User Jonathan with id = 7 logged in 7 times in 6 different days, five of them were consecutive days, so, Jonathan is an active user.
    
     
    
    Follow up: Could you write a general solution if the active users are those who logged in to their accounts for n or more consecutive days?


# JOINS

In [3]:
WITH ranked AS (
    SELECT 
        a. *
    FROM Logins a
    JOIN Logins b ON a.id = b.id AND DATEDIFF(b.login_date, a.login_date) = 1
    JOIN Logins c ON b.id = c.id AND DATEDIFF(c.login_date, b.login_date) = 1
    JOIN Logins d ON c.id = d.id AND DATEDIFF(d.login_date, c.login_date) = 1
    JOIN Logins e ON d.id = e.id AND DATEDIFF(e.login_date, d.login_date) = 1
)
SELECT 
    DISTINCT a.*
FROM Accounts a
JOIN ranked r
ON a.id = r.id
ORDER by a.id

# DENSE_RANK

In [ ]:
# Write your MySQL query statement below
with temp0 AS (
    SELECT  
        id,
        login_date,
        DENSE_RANK() OVER(PARTITION BY id ORDER BY login_date) AS row_num
    FROM Logins
),
temp1 AS (
    SELECT 
        id, 
        login_date, 
        row_num,
        DATE_ADD(login_date, INTERVAL -row_num DAY) as Groupings
    FROM temp0
),
answer_table AS (
    SELECT  
        id,
        Groupings,
        row_num,
        MIN(login_date) AS startDate,
        MAX(login_date) AS EndDate,
        COUNT(id),
        DATEDIFF(MAX(login_date), MIN(login_date)) AS duration
 FROM temp1
 GROUP BY id, Groupings
 HAVING datediff(MAX(login_date), MIN(login_date)) >= 4
 ORDER BY id, StartDate
) 
SELECT 
    DISTINCT acc.*
FROM Accounts acc
JOIN answer_table a 
ON acc.id = a.id
ORDER BY a.id